# Make the A-list

The A-list is the list of objects with all final products XID+ fluxes, redshift, and CIGALE physical properties.

Here we will loop over all the final catalogues, take the xid+ objects, and the set of columns present on every field. We will also add new 'best' photometry where for every common band type we take the valeu with the lowest error.

In [1]:
from herschelhelp_internal import git_version
print("This notebook was run with herschelhelp_internal version: \n{}".format(git_version()))
import datetime
print("This notebook was executed on: \n{}".format(datetime.datetime.now()))

This notebook was run with herschelhelp_internal version: 
1407877 (Mon Feb 4 12:56:29 2019 +0000)
This notebook was executed on: 
2020-10-27 19:16:35.205701


In [2]:
from astropy.table import Table, vstack
import glob
import yaml
import time
import os
import numpy as np
from herschelhelp_internal.utils import add_column_meta
SUFFIX = os.environ.get('SUFFIX', time.strftime("%Y%m%d"))
print(SUFFIX)

20201027


In [3]:
bands = ['u', 'g', 'r', 'i', 'z', 'y', 'j', 'h', 'k', 'ks']
always_keep = ['irac','mips','pacs','spire', 'best']

In [4]:
meta_files = glob.glob('../*/meta_main.yml')

In [5]:
meta_files

['../dmu32_COSMOS/meta_main.yml',
 '../dmu32_ELAIS-S1/meta_main.yml',
 '../dmu32_GAMA-09/meta_main.yml',
 '../dmu32_Lockman-SWIRE/meta_main.yml',
 '../dmu32_ELAIS-N2/meta_main.yml',
 '../dmu32_Herschel-Stripe-82/meta_main.yml',
 '../dmu32_SA13/meta_main.yml',
 '../dmu32_GAMA-12/meta_main.yml',
 '../dmu32_GAMA-15/meta_main.yml',
 '../dmu32_AKARI-SEP/meta_main.yml',
 '../dmu32_SGP/meta_main.yml',
 '../dmu32_xFLS/meta_main.yml',
 '../dmu32_CDFS-SWIRE/meta_main.yml',
 '../dmu32_SSDF/meta_main.yml',
 '../dmu32_XMM-LSS/meta_main.yml',
 '../dmu32_SPIRE-NEP/meta_main.yml',
 '../dmu32_AKARI-NEP/meta_main.yml',
 '../dmu32_ELAIS-N1/meta_main.yml',
 '../dmu32_NGP/meta_main.yml',
 '../dmu32_XMM-13hr/meta_main.yml',
 '../dmu32_Bootes/meta_main.yml',
 '../dmu32_HDF-N/meta_main.yml',
 '../dmu32_EGS/meta_main.yml']

In [6]:
meta_files[16:17]

['../dmu32_AKARI-NEP/meta_main.yml']

In [7]:
t =set()
t = t.union(set(('ads','wrty','adfg')))
t.add('sfgh')
t

{'adfg', 'ads', 'sfgh', 'wrty'}

In [8]:
def get_best_flux(table, band):
    "Return the lowest error non nan band "
    table=table.copy()
    f_best = np.full(len(table), np.nan)
    ferr_best = np.full(len(table), np.nan)
    m_best= np.full(len(table), np.nan)
    merr_best= np.full(len(table), np.nan)
    flag_best= np.full(len(table), False)
    camera = np.full(len(table), 'band_not_available')
    
    for col in table.colnames:
        if (col.startswith('m_') 
            and (not col.startswith('m_ap'))
            and col.endswith('_{}'.format(band))):
            #replace all nans
            m1 = np.isnan(f_best)
            f_best[m1] = table[m1]['f' + col[1:]]
            ferr_best[m1] = table[m1]['ferr' + col[1:]]
            m_best[m1] = table[m1][col]
            merr_best[m1] = table[m1]['merr' + col[1:]]
            try:
                flag_best[m1] = table[m1]['flag' + col[1:]]
            except:
                'NO FLAG COL FOR {}'.format(col)
            m1= m1 & (~np.isnan(table[col]))
            camera[m1] = np.full(np.sum(m1),col[2:])
            #replace all with larger flux error
            m2 = table['ferr' + col[1:]] < ferr_best
            f_best[m2] = table[m2]['f' + col[1:]]
            ferr_best[m2] = table[m2]['ferr' + col[1:]]
            m_best[m2] = table[m2][col]
            merr_best[m2] = table[m2]['merr' + col[1:]]
            try:
                flag_best[m2] = table[m2]['flag' + col[1:]]
            except:
                'NO FLAG COL FOR {}'.format(col)
            camera[m2] = np.full(np.sum(m2),col.split('_')[-2])
    
    return f_best, ferr_best, m_best, merr_best, flag_best, camera
    
def remove_specific_bands(table, bands_to_keep):
    "Return the table with all specific band fluxes removed"
    table = table.copy()
    cols_to_remove = set()
    for col in table.colnames:
        if col.startswith('m_ap_'):
            to_remove=(
                col,  
                'merr_ap' + col[4:],
                'f_ap' + col[4:],
                'ferr_ap' + col[4:]
            )
            cols_to_remove = cols_to_remove.union(set(to_remove))
    
    
    
    for col in table.colnames:
        if col.startswith('m_') and (col.split('_')[-2] not in bands_to_keep):
            print(col.split('_')[-2])
            to_remove =(
                col,  
                'merr' + col[1:],
                'f' + col[1:],
                'ferr' + col[1:],
                'flag' + col[1:]
                )
            cols_to_remove = cols_to_remove.union(set(to_remove))
            
    print('Removing ',cols_to_remove)
    for c in set(cols_to_remove):
        try:
            table.remove_columns([c])
        except:
            print('Column {} cannot be removed it is not there'.format(c))
            
            
            
    return table

In [9]:
full_cat = Table()
full_sources = Table()
for meta_file in meta_files: #meta_files[16:17]:
    meta = yaml.load(open(meta_file, 'r'))
    final_cat_loc = meta['final'].replace('dmu_products', '../..')
    #final_cat_loc = final_cat_loc.replace('_cigale','')
    
    final_cat = Table.read(final_cat_loc)
    final_cat = final_cat[final_cat['f_spire_500'] > 0.]
    sources = Table()
    sources['help_id'] = final_cat['help_id']
    
    for band in bands:
        f_best, ferr_best, m_best, merr_best, flag_best, camera = get_best_flux(final_cat, band)
        final_cat['f_best_{}'.format(band)] = f_best
        final_cat['ferr_best_{}'.format(band)] = ferr_best
        final_cat['m_best_{}'.format(band)] = m_best
        final_cat['merr_best_{}'.format(band)] = merr_best
        final_cat['flag_best_{}'.format(band)] = flag_best
        sources['best_{}'.format(band)] = camera
    
    
    final_cat = remove_specific_bands(final_cat,always_keep)
    
    full_cat = vstack([full_cat,final_cat])
    full_sources = vstack([full_sources,sources])

/Users/rs548/miniconda3/envs/herschelhelp_internal/lib/python3.6/site-packages/ipykernel_launcher.py:4: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  after removing the cwd from sys.path.


megacam
megacam
gpc1
gpc1
gpc1
gpc1
megacam
gpc1
gpc1
megacam
gpc1
gpc1
megacam
gpc1
gpc1
wircam
wircam
wircam
Removing  {'ferr_gpc1_g', 'm_ap_gpc1_r', 'f_megacam_u', 'f_wircam_j', 'ferr_ap_gpc1_i', 'flag_megacam_r', 'ferr_megacam_z', 'flag_ap_gpc1_z', 'f_ap_irac_i2', 'merr_megacam_u', 'merr_megacam_r', 'ferr_wircam_j', 'merr_ap_gpc1_r', 'f_gpc1_g', 'merr_gpc1_y', 'm_ap_gpc1_z', 'ferr_ap_irac_i1', 'm_ap_gpc1_g', 'ferr_gpc1_i', 'm_gpc1_g', 'f_megacam_r', 'ferr_ap_irac_i2', 'f_wircam_ks', 'm_megacam_g', 'flag_gpc1_z', 'ferr_wircam_y', 'ferr_ap_gpc1_y', 'm_megacam_z', 'merr_wircam_ks', 'merr_megacam_i', 'ferr_gpc1_r', 'merr_megacam_z', 'm_wircam_ks', 'flag_gpc1_i', 'flag_megacam_u', 'ferr_megacam_r', 'f_ap_gpc1_i', 'f_ap_gpc1_g', 'ferr_megacam_g', 'flag_gpc1_r', 'ferr_ap_gpc1_g', 'm_megacam_r', 'ferr_gpc1_y', 'merr_ap_gpc1_z', 'f_gpc1_i', 'f_megacam_z', 'flag_ap_gpc1_r', 'f_wircam_y', 'flag_gpc1_y', 'flag_megacam_z', 'flag_wircam_ks', 'merr_gpc1_i', 'flag_megacam_g', 'merr_wircam_j', 'fer

In [10]:
full_cat[:5]

help_id,field,ra,dec,hp_idx,ebv,redshift,zspec,f_irac_i1,ferr_irac_i1,m_irac_i1,merr_irac_i1,flag_irac_i1,f_irac_i2,ferr_irac_i2,m_irac_i2,merr_irac_i2,flag_irac_i2,f_mips_24,ferr_mips_24,flag_mips_24,f_pacs_green,ferr_pacs_green,f_pacs_red,ferr_pacs_red,f_spire_250,ferr_spire_250,flag_spire_250,f_spire_350,ferr_spire_350,flag_spire_350,f_spire_500,ferr_spire_500,flag_spire_500,cigale_mstar,cigale_mstar_err,cigale_sfr,cigale_sfr_err,cigale_dustlumin,cigale_dustlumin_err,cigale_dustlumin_ironly,cigale_dustlumin_ironly_err,cigale_chi2_opt,cigale_chi2_ir,cigale_chi2_red,cigale_chi2_ironly,stellarity,stellarity_origin,flag_cleaned,flag_merged,flag_gaia,flag_optnir_obs,flag_optnir_det,zspec_qual,zspec_association_flag,f_best_u,ferr_best_u,m_best_u,merr_best_u,flag_best_u,f_best_g,ferr_best_g,m_best_g,merr_best_g,flag_best_g,f_best_r,ferr_best_r,m_best_r,merr_best_r,flag_best_r,f_best_i,ferr_best_i,m_best_i,merr_best_i,flag_best_i,f_best_z,ferr_best_z,m_best_z,merr_best_z,flag_best_z,f_best_y,ferr_best_y,m_best_y,merr_best_y,flag_best_y,f_best_j,ferr_best_j,m_best_j,merr_best_j,flag_best_j,f_best_h,ferr_best_h,m_best_h,merr_best_h,flag_best_h,f_best_k,ferr_best_k,m_best_k,merr_best_k,flag_best_k,f_best_ks,ferr_best_ks,m_best_ks,merr_best_ks,flag_best_ks
,,deg,deg,,,,,uJy,uJy,mag,mag,,uJy,uJy,mag,mag,,uJy,uJy,,uJy,uJy,uJy,uJy,uJy,uJy,,uJy,uJy,,uJy,uJy,,solMass,solMass,solMass / yr,solMass / yr,W,W,W,W,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
bytes27,bytes18,float64,float64,int64,float64,float32,float64,float32,float32,float32,float32,bool,float32,float32,float32,float32,bool,float32,float32,bool,float32,float32,float32,float32,float32,float32,bool,float32,float32,bool,float32,float32,bool,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float32,bytes20,bool,bool,int64,int64,int64,int64,bool,float64,float64,float64,float64,bool,float64,float64,float64,float64,bool,float64,float64,float64,float64,bool,float64,float64,float64,float64,bool,float64,float64,float64,float64,bool,float64,float64,float64,float64,bool,float64,float64,float64,float64,bool,float64,float64,float64,float64,bool,float64,float64,float64,float64,bool,float64,float64,float64,float64,bool
HELP_J174958.317+661836.745,AKARI-NEP,267.492987431287,66.3102069321774,190145243,0.033387688106114165,0.554365,nan,45.20224,2.2023754,19.7621,0.0529,False,33.83454,1.9009285,20.0766,0.061,False,402.1716,595.37256,True,nan,nan,nan,nan,2121.0498,3132.9673,True,2661.1948,3147.8647,True,5823.619,5904.3584,True,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.07,irac_stellarity,False,False,0,5,5,-99,False,nan,nan,nan,nan,False,nan,nan,nan,nan,False,5.004949724002007,0.7662706259906309,22.151500701904297,0.1662289947271347,False,13.118362834413999,0.8155901192139533,21.105300903320312,0.06750199943780899,False,20.97393124912173,4.2283169924751745,20.595800399780273,0.2188829928636551,False,30.854588817745707,7.23153243970557,20.176700592041016,0.25446900725364685,False,nan,nan,nan,nan,False,nan,nan,nan,nan,False,nan,nan,nan,nan,False,nan,nan,nan,nan,False
HELP_J174958.976+661841.815,AKARI-NEP,267.495734291287,66.3116153421774,190145243,0.03337325314561728,nan,nan,26.198723,2.1258457,20.3543,0.0881,False,12.884861,1.7896034,21.1248,0.1508,False,336.59177,525.26245,True,nan,nan,nan,nan,1335.3774,1936.1816,True,1689.2697,2537.146,True,3819.2466,5055.3735,True,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.2,irac_stellarity,False,False,0,5,5,-99,False,nan,nan,nan,nan,False,1.6530278930647855,0.7581522520049238,23.354299545288086,0.497967004776001,False,6.85173130489252,0.5283551807900024,21.81049919128418,0.0837239995598793,False,15.51530055228207,1.0254447203448556,20.923099517822266,0.07175900042057037,False,27.509327932148043,2.1780214833164258,20.301300048828125,0.08596199750900269,False,22.00494534359914,3.9840047636574916,20.543699264526367,0.19657300412654877,False,nan,nan,nan,nan,False,nan,nan,nan,nan,False,nan,na

In [11]:
len(full_cat)

31442

In [12]:
len(full_cat.colnames)

105

In [17]:
full_cat.colnames

['help_id',
 'field',
 'ra',
 'dec',
 'hp_idx',
 'ebv',
 'redshift',
 'zspec',
 'f_irac_i1',
 'ferr_irac_i1',
 'm_irac_i1',
 'merr_irac_i1',
 'flag_irac_i1',
 'f_irac_i2',
 'ferr_irac_i2',
 'm_irac_i2',
 'merr_irac_i2',
 'flag_irac_i2',
 'f_mips_24',
 'ferr_mips_24',
 'flag_mips_24',
 'f_pacs_green',
 'ferr_pacs_green',
 'f_pacs_red',
 'ferr_pacs_red',
 'f_spire_250',
 'ferr_spire_250',
 'flag_spire_250',
 'f_spire_350',
 'ferr_spire_350',
 'flag_spire_350',
 'f_spire_500',
 'ferr_spire_500',
 'flag_spire_500',
 'cigale_mstar',
 'cigale_mstar_err',
 'cigale_sfr',
 'cigale_sfr_err',
 'cigale_dustlumin',
 'cigale_dustlumin_err',
 'cigale_dustlumin_ironly',
 'cigale_dustlumin_ironly_err',
 'cigale_chi2_opt',
 'cigale_chi2_ir',
 'cigale_chi2_red',
 'cigale_chi2_ironly',
 'stellarity',
 'stellarity_origin',
 'flag_cleaned',
 'flag_merged',
 'flag_gaia',
 'flag_optnir_obs',
 'flag_optnir_det',
 'zspec_qual',
 'zspec_association_flag',
 'f_best_u',
 'ferr_best_u',
 'm_best_u',
 'merr_best_u',

In [14]:
final_cat = Table.read(final_cat_loc)
final_cat = final_cat[final_cat['f_spire_500']>0]
final_cat['help_id', 'f_gpc1_g','ferr_gpc1_g', 'f_megacam_g', 'f_megacam_u'][:5]

help_id,f_gpc1_g,ferr_gpc1_g,f_megacam_g,f_megacam_u
,uJy,uJy,uJy,uJy
bytes27,float64,float64,float32,float32
HELP_J174958.317+661836.745,nan,nan,nan,nan
HELP_J174958.976+661841.815,1.6530278930647855,0.7581522520049238,nan,nan
HELP_J174959.565+661835.133,8.53885598194625,1.475520521644639,nan,nan
HELP_J175001.532+661913.701,2.109598602566689,0.6096316888008567,nan,nan
HELP_J175001.906+661938.012,4.365161389769828,0.3500013290568288,nan,nan


In [15]:
full_sources[:5]

help_id,best_u,best_g,best_r,best_i,best_z,best_y,best_j,best_h,best_k,best_ks
bytes27,str18,str18,str18,str18,str18,str18,str18,str18,str18,str18
HELP_J174958.317+661836.745,band_not_available,band_not_available,gpc1_r,gpc1_i,gpc1_z,gpc1_y,band_not_available,band_not_available,band_not_available,band_not_available
HELP_J174958.976+661841.815,band_not_available,gpc1_g,gpc1_r,gpc1_i,gpc1_z,gpc1_y,band_not_available,band_not_available,band_not_available,band_not_available
HELP_J174959.565+661835.133,band_not_available,gpc1_g,gpc1_r,gpc1_i,gpc1_z,gpc1_y,band_not_available,band_not_available,band_not_available,band_not_available
HELP_J175001.532+661913.701,band_not_available,gpc1_g,gpc1_r,gpc1_i,gpc1_z,gpc1_y,band_not_available,band_not_available,band_not_available,band_not_available
HELP_J175001.906+661938.012,band_not_available,gpc1_g,gpc1_r,gpc1_i,gpc1_z,gpc1_y,band_not_available,band_not_available,band_not_available,band_not_available


In [16]:
!mkdir data
full_cat = add_column_meta(full_cat, 'columns.yml')
full_cat.write('./data/HELP_all_sky_A-list_{}.fits'.format(SUFFIX), overwrite=True)
#full_cat.write('./data/HELP_all_sky_xid_A-list_{}.csv'.format(SUFFIX), overwrite=True)
full_sources.write('./data/HELP_all_sky_xid_objects_sources_{}.fits'.format(SUFFIX), overwrite=True)
#full_sources.write('./data/HELP_all_sky_xid_objects_sources_{}.csv'.format(SUFFIX), overwrite=True)

mkdir: data: File exists


/Users/rs548/GitHub/herschelhelp_internal/herschelhelp_internal/utils.py:357: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  meta = yaml.load(open(column_yml_file, "r"))
